# Crypto HFT Trading Strategy - Model Development

This notebook implements machine learning models for high-frequency trading prediction, following the SGX methodology with ensemble approaches and rolling model selection.

## Objectives:
1. Train multiple ML models on engineered features
2. Implement ensemble methods and model stacking
3. Perform hyperparameter optimization
4. Evaluate model performance with proper time-series validation
5. Implement rolling model selection for live trading

In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    ExtraTreesClassifier, ExtraTreesRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    AdaBoostClassifier, AdaBoostRegressor
)
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

# XGBoost and LightGBM
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available")
    XGBOOST_AVAILABLE = False

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    print("LightGBM not available")
    LIGHTGBM_AVAILABLE = False

# Model evaluation and tuning
from sklearn.model_selection import (
    GridSearchCV, RandomizedSearchCV, TimeSeriesSplit,
    cross_val_score, validation_curve
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report, confusion_matrix,
    mean_squared_error, mean_absolute_error, r2_score
)

# Import project modules
from models.ml_models import MLModelManager
from models.model_selection import RollingModelSelection
from utils.visualization import OrderBookVisualizer
from utils.metrics import PerformanceMetrics

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Load Processed Data

In [ ]:
# Load processed features and targets
print("Loading processed data...")

try:
    X_train = pd.read_parquet('../data/processed/X_train_scaled.parquet')
    X_val = pd.read_parquet('../data/processed/X_val_scaled.parquet')
    X_test = pd.read_parquet('../data/processed/X_test_scaled.parquet')
    
    y_train = pd.read_parquet('../data/processed/y_train.parquet')
    y_val = pd.read_parquet('../data/processed/y_val.parquet')
    y_test = pd.read_parquet('../data/processed/y_test.parquet')
    
    # Load metadata
    import json
    with open('../data/processed/feature_metadata.json', 'r') as f:
        metadata = json.load(f)
    
    print(f"✓ Data loaded successfully")
    print(f"  Training: {X_train.shape}")
    print(f"  Validation: {X_val.shape}")
    print(f"  Test: {X_test.shape}")
    print(f"  Target variables: {y_train.shape[1]}")
    
except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please run 02_feature_engineering.ipynb first.")
    raise

# Display target variables
print(f"\nAvailable target variables:")
for i, col in enumerate(y_train.columns):
    print(f"  {i+1:2d}. {col}")

## 2. Model Configuration and Setup

In [ ]:
# Initialize model manager
model_manager = MLModelManager()

# Define model configurations inspired by SGX methodology
classification_models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [5, 10, 15, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        }
    },
    'ExtraTrees': {
        'model': ExtraTreesClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [5, 10, 15, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    'SVM': {
        'model': SVC(probability=True, random_state=42),
        'params': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
            'kernel': ['rbf', 'linear']
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga']
        }
    },
    'MLP': {
        'model': MLPClassifier(random_state=42, max_iter=500),
        'params': {
            'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['constant', 'adaptive']
        }
    }
}

# Add XGBoost if available
if XGBOOST_AVAILABLE:
    classification_models['XGBoost'] = {
        'model': xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        }
    }

# Add LightGBM if available
if LIGHTGBM_AVAILABLE:
    classification_models['LightGBM'] = {
        'model': lgb.LGBMClassifier(random_state=42, verbose=-1),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        }
    }

print(f"Configured {len(classification_models)} classification models:")
for name in classification_models.keys():
    print(f"  - {name}")

## 3. Model Training and Validation Pipeline

In [ ]:
def train_and_evaluate_model(model, param_grid, X_train, y_train, X_val, y_val, 
                           model_name, target_name, cv_folds=3):
    """
    Train and evaluate a single model with hyperparameter tuning
    """
    print(f"\nTraining {model_name} for {target_name}...")
    
    # Time series cross-validation
    tscv = TimeSeriesSplit(n_splits=cv_folds)
    
    # Randomized search for efficiency (use subset of parameter combinations)
    search = RandomizedSearchCV(
        model, param_grid, cv=tscv, scoring='accuracy',
        n_iter=20, random_state=42, n_jobs=-1, verbose=0
    )
    
    # Fit the model
    search.fit(X_train, y_train)
    
    # Best model
    best_model = search.best_estimator_
    
    # Predictions
    train_pred = best_model.predict(X_train)
    val_pred = best_model.predict(X_val)
    
    # Probabilities for classification
    if hasattr(best_model, 'predict_proba'):
        train_proba = best_model.predict_proba(X_train)[:, 1]
        val_proba = best_model.predict_proba(X_val)[:, 1]
    else:
        train_proba = train_pred
        val_proba = val_pred
    
    # Calculate metrics
    results = {
        'model_name': model_name,
        'target_name': target_name,
        'best_params': search.best_params_,
        'best_cv_score': search.best_score_,
        'train_accuracy': accuracy_score(y_train, train_pred),
        'val_accuracy': accuracy_score(y_val, val_pred),
        'train_f1': f1_score(y_train, train_pred, average='weighted'),
        'val_f1': f1_score(y_val, val_pred, average='weighted'),
        'model': best_model,
        'train_pred': train_pred,
        'val_pred': val_pred,
        'train_proba': train_proba,
        'val_proba': val_proba
    }
    
    # Add AUC if binary classification
    if len(np.unique(y_train)) == 2:
        results['train_auc'] = roc_auc_score(y_train, train_proba)
        results['val_auc'] = roc_auc_score(y_val, val_proba)
    
    print(f"  ✓ CV Score: {search.best_score_:.4f}")
    print(f"  ✓ Val Accuracy: {results['val_accuracy']:.4f}")
    print(f"  ✓ Val F1: {results['val_f1']:.4f}")
    
    return results

# Select target variables to train on
priority_targets = [
    'ETH_direction_1',
    'ETH_direction_5',
    'XBT_direction_1',
    'XBT_direction_5',
    'ETH_outperforms_1',
    'ETH_outperforms_5'
]

print(f"Training models on {len(priority_targets)} priority targets...")
print(f"Targets: {priority_targets}")

## 4. Model Training Execution

In [ ]:
# Storage for all results
all_results = []
trained_models = {}

# Train models for each target
for target in priority_targets:
    print(f"\n{'='*60}")
    print(f"TRAINING MODELS FOR TARGET: {target}")
    print(f"{'='*60}")
    
    # Get target data (drop NaN values)
    y_train_target = y_train[target].dropna()
    y_val_target = y_val[target].dropna()
    
    # Align features with target
    X_train_target = X_train.loc[y_train_target.index]
    X_val_target = X_val.loc[y_val_target.index]
    
    print(f"Training samples: {len(X_train_target)}")
    print(f"Validation samples: {len(X_val_target)}")
    print(f"Target distribution: {y_train_target.value_counts().to_dict()}")
    
    target_results = []
    target_models = {}
    
    # Train each model
    for model_name, config in classification_models.items():
        try:
            result = train_and_evaluate_model(
                model=config['model'],
                param_grid=config['params'],
                X_train=X_train_target,
                y_train=y_train_target,
                X_val=X_val_target,
                y_val=y_val_target,
                model_name=model_name,
                target_name=target
            )
            
            target_results.append(result)
            target_models[model_name] = result['model']
            all_results.append(result)
            
        except Exception as e:
            print(f"  ✗ Error training {model_name}: {str(e)}")
            continue
    
    trained_models[target] = target_models
    
    # Show top performers for this target
    if target_results:
        target_df = pd.DataFrame([{k: v for k, v in r.items() if k not in ['model', 'train_pred', 'val_pred', 'train_proba', 'val_proba']} 
                                for r in target_results])
        target_df = target_df.sort_values('val_accuracy', ascending=False)
        
        print(f"\nTop 3 performers for {target}:")
        for i, (_, row) in enumerate(target_df.head(3).iterrows()):
            print(f"  {i+1}. {row['model_name']}: {row['val_accuracy']:.4f} accuracy, {row['val_f1']:.4f} F1")

print(f"\n{'='*60}")
print(f"MODEL TRAINING COMPLETED")
print(f"Total models trained: {len(all_results)}")
print(f"{'='*60}")

## 5. Model Performance Analysis

In [ ]:
# Create comprehensive results dataframe
results_df = pd.DataFrame([{k: v for k, v in r.items() 
                          if k not in ['model', 'train_pred', 'val_pred', 'train_proba', 'val_proba']} 
                         for r in all_results])

print("MODEL PERFORMANCE SUMMARY")
print("="*50)

# Overall best performers
print("\nTop 10 Models by Validation Accuracy:")
top_models = results_df.nlargest(10, 'val_accuracy')
for i, (_, row) in enumerate(top_models.iterrows()):
    print(f"{i+1:2d}. {row['model_name']:15s} ({row['target_name']:20s}): {row['val_accuracy']:.4f}")

# Model comparison by target
print("\nAverage Performance by Model Type:")
model_avg = results_df.groupby('model_name')[['val_accuracy', 'val_f1']].mean().sort_values('val_accuracy', ascending=False)
print(model_avg.round(4))

# Target difficulty analysis
print("\nAverage Performance by Target:")
target_avg = results_df.groupby('target_name')[['val_accuracy', 'val_f1']].mean().sort_values('val_accuracy', ascending=False)
print(target_avg.round(4))

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Model performance heatmap
pivot_acc = results_df.pivot(index='model_name', columns='target_name', values='val_accuracy')
sns.heatmap(pivot_acc, annot=True, fmt='.3f', cmap='viridis', ax=axes[0, 0])
axes[0, 0].set_title('Validation Accuracy by Model and Target')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. F1 score heatmap
pivot_f1 = results_df.pivot(index='model_name', columns='target_name', values='val_f1')
sns.heatmap(pivot_f1, annot=True, fmt='.3f', cmap='plasma', ax=axes[0, 1])
axes[0, 1].set_title('Validation F1 Score by Model and Target')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Model ranking distribution
model_counts = results_df['model_name'].value_counts()
axes[1, 0].bar(model_counts.index, model_counts.values)
axes[1, 0].set_title('Number of Targets per Model')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].set_ylabel('Count')

# 4. Accuracy distribution
for model in results_df['model_name'].unique():
    model_data = results_df[results_df['model_name'] == model]['val_accuracy']
    axes[1, 1].hist(model_data, alpha=0.6, label=model, bins=10)

axes[1, 1].set_title('Validation Accuracy Distribution by Model')
axes[1, 1].set_xlabel('Validation Accuracy')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## 6. Feature Importance Analysis

In [ ]:
# Feature importance analysis for tree-based models
def extract_feature_importance(models_dict, feature_names):
    """
    Extract and aggregate feature importance from tree-based models
    """
    importance_data = []
    
    for target, models in models_dict.items():
        for model_name, model in models.items():
            if hasattr(model, 'feature_importances_'):
                importances = model.feature_importances_
                for i, (feature, importance) in enumerate(zip(feature_names, importances)):
                    importance_data.append({
                        'target': target,
                        'model': model_name,
                        'feature': feature,
                        'importance': importance,
                        'rank': i
                    })
    
    return pd.DataFrame(importance_data)

# Extract feature importance
feature_importance_df = extract_feature_importance(trained_models, X_train.columns)

if not feature_importance_df.empty:
    # Aggregate importance across models and targets
    agg_importance = feature_importance_df.groupby('feature')['importance'].agg(['mean', 'std', 'count']).reset_index()
    agg_importance = agg_importance.sort_values('mean', ascending=False)
    
    print("TOP 20 MOST IMPORTANT FEATURES:")
    print("="*60)
    for i, (_, row) in enumerate(agg_importance.head(20).iterrows()):
        print(f"{i+1:2d}. {row['feature']:40s} {row['mean']:.6f} (±{row['std']:.6f})")
    
    # Plot top features
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Top 20 features
    top_20 = agg_importance.head(20)
    axes[0].barh(range(len(top_20)), top_20['mean'])
    axes[0].set_yticks(range(len(top_20)))
    axes[0].set_yticklabels(top_20['feature'], fontsize=8)
    axes[0].set_xlabel('Average Feature Importance')
    axes[0].set_title('Top 20 Features by Average Importance')
    axes[0].invert_yaxis()
    
    # Feature importance by category
    feature_categories = {
        'ETH': [f for f in agg_importance['feature'] if f.startswith('ETH_')],
        'XBT': [f for f in agg_importance['feature'] if f.startswith('XBT_')],
        'Cross-Asset': [f for f in agg_importance['feature'] if any(x in f for x in ['correlation', 'ratio', 'differential', 'lead'])],
        'Time': [f for f in agg_importance['feature'] if any(x in f for x in ['hour', 'minute', 'session'])],
        'Technical': [f for f in agg_importance['feature'] if any(x in f for x in ['rsi', 'macd', 'bb_', 'momentum'])]
    }
    
    category_importance = {}
    for category, features in feature_categories.items():
        category_features = agg_importance[agg_importance['feature'].isin(features)]
        if not category_features.empty:
            category_importance[category] = category_features['mean'].sum()
    
    if category_importance:
        categories = list(category_importance.keys())
        importance_values = list(category_importance.values())
        
        axes[1].pie(importance_values, labels=categories, autopct='%1.1f%%', startangle=90)
        axes[1].set_title('Feature Importance by Category')
    
    plt.tight_layout()
    plt.show()
else:
    print("No feature importance data available (no tree-based models trained successfully)")

## 7. Ensemble Model Creation

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression as LR

def create_ensemble_models(trained_models_dict, X_train, y_train, X_val, y_val):
    """
    Create ensemble models from the best performing individual models
    """
    ensemble_results = {}
    
    for target, models in trained_models_dict.items():
        if len(models) < 2:
            continue
            
        print(f"\nCreating ensemble for {target}...")
        
        # Get aligned data
        y_train_target = y_train[target].dropna()
        y_val_target = y_val[target].dropna()
        X_train_target = X_train.loc[y_train_target.index]
        X_val_target = X_val.loc[y_val_target.index]
        
        # Select top 3 models for ensemble
        target_results = [r for r in all_results if r['target_name'] == target]
        top_3_models = sorted(target_results, key=lambda x: x['val_accuracy'], reverse=True)[:3]
        
        # Create voting classifier
        estimators = [(result['model_name'], models[result['model_name']]) 
                     for result in top_3_models if result['model_name'] in models]
        
        if len(estimators) >= 2:
            # Hard voting ensemble
            hard_ensemble = VotingClassifier(estimators=estimators, voting='hard')
            hard_ensemble.fit(X_train_target, y_train_target)
            
            # Soft voting ensemble (if all models support predict_proba)
            try:
                soft_ensemble = VotingClassifier(estimators=estimators, voting='soft')
                soft_ensemble.fit(X_train_target, y_train_target)
                
                # Evaluate both ensembles
                hard_pred = hard_ensemble.predict(X_val_target)
                soft_pred = soft_ensemble.predict(X_val_target)
                
                hard_acc = accuracy_score(y_val_target, hard_pred)
                soft_acc = accuracy_score(y_val_target, soft_pred)
                
                ensemble_results[target] = {
                    'hard_ensemble': hard_ensemble,
                    'soft_ensemble': soft_ensemble,
                    'hard_accuracy': hard_acc,
                    'soft_accuracy': soft_acc,
                    'component_models': [name for name, _ in estimators]
                }
                
                print(f"  Hard Voting Accuracy: {hard_acc:.4f}")
                print(f"  Soft Voting Accuracy: {soft_acc:.4f}")
                print(f"  Component Models: {[name for name, _ in estimators]}")
                
            except Exception as e:
                print(f"  Soft voting failed: {e}")
                # Fallback to hard voting only
                hard_pred = hard_ensemble.predict(X_val_target)
                hard_acc = accuracy_score(y_val_target, hard_pred)
                
                ensemble_results[target] = {
                    'hard_ensemble': hard_ensemble,
                    'hard_accuracy': hard_acc,
                    'component_models': [name for name, _ in estimators]
                }
                
                print(f"  Hard Voting Accuracy: {hard_acc:.4f}")
    
    return ensemble_results

# Create ensemble models
print("CREATING ENSEMBLE MODELS")
print("="*40)
ensemble_models = create_ensemble_models(trained_models, X_train, y_train, X_val, y_val)

print(f"\nCreated ensembles for {len(ensemble_models)} targets")

## 8. Rolling Model Selection Implementation

In [ ]:
# Initialize rolling model selection
rolling_selector = RollingModelSelection(
    models=list(classification_models.keys()),
    window_size=1000,  # 1000 samples per window
    retrain_frequency=100  # Retrain every 100 samples
)

def simulate_rolling_model_selection(target_name, X_data, y_data, test_size=500):
    """
    Simulate rolling model selection for a specific target
    """
    print(f"\nSimulating rolling selection for {target_name}...")
    
    # Get target data
    y_target = y_data[target_name].dropna()
    X_target = X_data.loc[y_target.index]
    
    # Take last test_size samples for simulation
    if len(X_target) > test_size:
        X_sim = X_target.iloc[-test_size:]
        y_sim = y_target.iloc[-test_size:]
    else:
        X_sim = X_target
        y_sim = y_target
    
    print(f"  Simulation samples: {len(X_sim)}")
    
    # Initialize tracking variables
    predictions = []
    model_selections = []
    performance_history = []
    
    # Rolling simulation
    window_size = min(200, len(X_sim) // 3)  # Adaptive window size
    
    for i in range(window_size, len(X_sim), 10):  # Step by 10 for efficiency
        # Training window
        X_window = X_sim.iloc[i-window_size:i]
        y_window = y_sim.iloc[i-window_size:i]
        
        # Next sample to predict
        if i < len(X_sim):
            X_next = X_sim.iloc[[i]]
            y_next = y_sim.iloc[i]
            
            # Train a simple model for this window (for speed)
            from sklearn.ensemble import RandomForestClassifier
            model = RandomForestClassifier(n_estimators=50, random_state=42)
            model.fit(X_window, y_window)
            
            # Make prediction
            pred = model.predict(X_next)[0]
            predictions.append(pred)
            
            # Calculate accuracy for this window
            window_pred = model.predict(X_window)
            window_acc = accuracy_score(y_window, window_pred)
            
            performance_history.append({
                'step': i,
                'window_accuracy': window_acc,
                'prediction': pred,
                'actual': y_next,
                'correct': pred == y_next
            })
    
    # Calculate overall performance
    if performance_history:
        overall_accuracy = np.mean([p['correct'] for p in performance_history])
        print(f"  Rolling prediction accuracy: {overall_accuracy:.4f}")
        
        return {
            'target': target_name,
            'accuracy': overall_accuracy,
            'predictions': len(predictions),
            'performance_history': performance_history
        }
    
    return None

# Simulate rolling selection for key targets
rolling_results = []
test_targets = ['ETH_direction_1', 'ETH_direction_5', 'XBT_direction_1']

print("ROLLING MODEL SELECTION SIMULATION")
print("="*50)

for target in test_targets:
    result = simulate_rolling_model_selection(target, X_val, y_val)
    if result:
        rolling_results.append(result)

print(f"\nCompleted rolling simulation for {len(rolling_results)} targets")

## 9. Model Persistence and Export

In [ ]:
# Save trained models and results with error handling
import os
os.makedirs('../models/trained', exist_ok=True)
os.makedirs('../models/ensembles', exist_ok=True)
try:
    # Save individual models
    for target, models in trained_models.items():
        for model_name, model in models.items():
            filename = f"../models/trained/{target}_{model_name}.pkl"
            joblib.dump(model, filename)
    # Save ensemble models
    for target, ensemble_data in ensemble_models.items():
        if 'hard_ensemble' in ensemble_data:
            joblib.dump(ensemble_data['hard_ensemble'], f"../models/ensembles/{target}_hard_ensemble.pkl")
        if 'soft_ensemble' in ensemble_data:
            joblib.dump(ensemble_data['soft_ensemble'], f"../models/ensembles/{target}_soft_ensemble.pkl")
    # Save results and metadata
    results_df.to_csv('../data/processed/model_training_results.csv', index=False)
    results_df.to_parquet('../data/processed/model_training_results.parquet')
    with open('../models/model_registry.json', 'w') as f:
        json.dump(model_registry, f, indent=2)
    with open('../models/ensemble_registry.json', 'w') as f:
        json.dump(ensemble_registry, f, indent=2)
    with open('../data/processed/training_summary.json', 'w') as f:
        json.dump(results_summary, f, indent=2, default=str)
    if not feature_importance_df.empty:
        feature_importance_df.to_csv('../data/processed/feature_importance.csv', index=False)
        agg_importance.to_csv('../data/processed/aggregated_feature_importance.csv', index=False)
    print("✓ Model persistence completed. All files saved.")
except Exception as e:
    print(f"Error saving models or results: {e}")

## 10. Model Development Summary

In [ ]:
print("MODEL DEVELOPMENT SUMMARY")
print("="*70)

print(f"\n1. TRAINING STATISTICS:")
print(f"   Total models trained: {len(all_results)}")
print(f"   Model types: {len(classification_models)}")
print(f"   Target variables: {len(set([r['target_name'] for r in all_results]))}")
print(f"   Ensemble models created: {len(ensemble_models)}")

print(f"\n2. PERFORMANCE HIGHLIGHTS:")
if not results_df.empty:
    best_model = results_df.loc[results_df['val_accuracy'].idxmax()]
    print(f"   Best single model: {best_model['model_name']} on {best_model['target_name']}")
    print(f"   Best accuracy: {best_model['val_accuracy']:.4f}")
    print(f"   Average accuracy: {results_df['val_accuracy'].mean():.4f}")
    print(f"   Accuracy std: {results_df['val_accuracy'].std():.4f}")

print(f"\n3. MODEL TYPE PERFORMANCE:")
for model_type in model_avg.index:
    print(f"   {model_type:15s}: {model_avg.loc[model_type, 'val_accuracy']:.4f} avg accuracy")

print(f"\n4. TARGET DIFFICULTY RANKING:")
for target in target_avg.index:
    print(f"   {target:25s}: {target_avg.loc[target, 'val_accuracy']:.4f} avg accuracy")

if ensemble_models:
    print(f"\n5. ENSEMBLE PERFORMANCE:")
    for target, ensemble_data in ensemble_models.items():
        if 'soft_accuracy' in ensemble_data:
            print(f"   {target:25s}: {ensemble_data['soft_accuracy']:.4f} (soft voting)")
        else:
            print(f"   {target:25s}: {ensemble_data['hard_accuracy']:.4f} (hard voting)")

if rolling_results:
    print(f"\n6. ROLLING SIMULATION RESULTS:")
    for result in rolling_results:
        print(f"   {result['target']:20s}: {result['accuracy']:.4f} accuracy ({result['predictions']} predictions)")

print(f"\n7. FEATURE INSIGHTS:")
if not feature_importance_df.empty:
    print(f"   Total features analyzed: {len(agg_importance)}")
    print(f"   Top feature: {agg_importance.iloc[0]['feature']}")
    print(f"   Top importance: {agg_importance.iloc[0]['mean']:.6f}")
    
    # Feature category insights
    eth_features = len([f for f in agg_importance['feature'] if f.startswith('ETH_')])
    xbt_features = len([f for f in agg_importance['feature'] if f.startswith('XBT_')])
    cross_features = len([f for f in agg_importance['feature'] if any(x in f for x in ['correlation', 'ratio', 'differential'])])
    
    print(f"   ETH-specific features: {eth_features}")
    print(f"   XBT-specific features: {xbt_features}")
    print(f"   Cross-asset features: {cross_features}")

print(f"\n8. NEXT STEPS:")
print(f"   ✓ Models ready for strategy backtesting")
print(f"   ✓ Ensemble models available for improved predictions")
print(f"   ✓ Rolling selection framework implemented")
print(f"   ✓ Feature importance analysis completed")
print(f"   → Ready for strategy implementation and backtesting")

print(f"\n" + "="*70)
print(f"MODEL DEVELOPMENT COMPLETED SUCCESSFULLY!")
print(f"All models and results saved to respective directories.")
print(f"="*70)

In [ ]:
# Set global seed for reproducibility
import random
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
try:
    import os
    os.environ["PYTHONHASHSEED"] = str(SEED)
except Exception:
    pass

In [ ]:
# Assert required columns exist before model training
assert X_train.shape[0] > 0 and X_val.shape[0] > 0 and X_test.shape[0] > 0, "Feature datasets are empty!"
assert y_train.shape[0] > 0 and y_val.shape[0] > 0 and y_test.shape[0] > 0, "Target datasets are empty!"

In [ ]:
# Save trained models and results with error handling
import os
os.makedirs('../models/trained', exist_ok=True)
os.makedirs('../models/ensembles', exist_ok=True)
try:
    # Save individual models
    for target, models in trained_models.items():
        for model_name, model in models.items():
            filename = f"../models/trained/{target}_{model_name}.pkl"
            joblib.dump(model, filename)
    # Save ensemble models
    for target, ensemble_data in ensemble_models.items():
        if 'hard_ensemble' in ensemble_data:
            joblib.dump(ensemble_data['hard_ensemble'], f"../models/ensembles/{target}_hard_ensemble.pkl")
        if 'soft_ensemble' in ensemble_data:
            joblib.dump(ensemble_data['soft_ensemble'], f"../models/ensembles/{target}_soft_ensemble.pkl")
    # Save results and metadata
    results_df.to_csv('../data/processed/model_training_results.csv', index=False)
    results_df.to_parquet('../data/processed/model_training_results.parquet')
    with open('../models/model_registry.json', 'w') as f:
        json.dump(model_registry, f, indent=2)
    with open('../models/ensemble_registry.json', 'w') as f:
        json.dump(ensemble_registry, f, indent=2)
    with open('../data/processed/training_summary.json', 'w') as f:
        json.dump(results_summary, f, indent=2, default=str)
    if not feature_importance_df.empty:
        feature_importance_df.to_csv('../data/processed/feature_importance.csv', index=False)
        agg_importance.to_csv('../data/processed/aggregated_feature_importance.csv', index=False)
    print("✓ Model persistence completed. All files saved.")
except Exception as e:
    print(f"Error saving models or results: {e}")

In [ ]:
# Simple output validation (unit-test style)
assert results_df.shape[0] > 0, "No model results generated!"
if not feature_importance_df.empty:
    assert agg_importance.shape[0] > 0, "No feature importance aggregated!"
print("[TESTS PASSED] Key outputs are valid.")